In [1]:
//CONTEXTE DE TRAVAIL
import org.apache.spark.sql.SparkSession
import org.apache.spark._
import org.apache.spark.rdd._
import org.apache.spark.SparkContext._
import org.apache.spark.mllib.feature.HashingTF
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.tree.GradientBoostedTrees
import org.apache.spark.mllib.tree.configuration.BoostingStrategy
import scala.util.{Success, Try}

//LECTURE DU FICHIER D'ENTREE DES COMMENTAIRES AVEC DELIMITEUR #
val commentaires_bruts = spark.read.option("delimiter", "#").option("multiline",true).csv("comment_fr_modele.csv")

//CREATION DE LA TABLE ASSOCIEE AUX COMMENTAIRES LUS
commentaires_bruts.createOrReplaceTempView("commentaires_bruts")

//CREATION DU NOM DES COLONNES POUR LA TABLE COMMENTAIRES
val commentaires = spark.sql("select _c0 as Id_comment, _c1 as commentaire, _c2 as qualite from commentaires_bruts")

//CREATION DE LA TABLE ASSOCIEE AUX COMMENTAIRES FINALISES
commentaires.createOrReplaceTempView("commentaires")


commentaires_bruts = [_c0: string, _c1: string ... 1 more field]
commentaires = [Id_comment: string, commentaire: string ... 1 more field]


[Id_comment: string, commentaire: string ... 1 more field]

In [2]:
val commentaires_negatifs = spark.sql("select * from commentaires where qualite = 0") 
val commentaires_positifs = spark.sql("select * from commentaires where qualite = 1")
val commentaires_positifs_nbre = commentaires_positifs.count()
val commentaires_negatifs_nbre = commentaires_negatifs.count()


//DEFINITION DE LA TAILLE DE L'ECHANTILLON D'APPRENTISSAGE
val taille_echantillon = Math.min(commentaires_negatifs_nbre, commentaires_positifs_nbre).toInt
//DEFINITION DE L'ECHANTILLON D'APPRENTISSAGE AVEC AUTANT DE COMMENTAIRES POSITIFS QUE NEGATIFS
var echantillon = commentaires_positifs.limit(taille_echantillon).unionAll(commentaires_negatifs.limit(taille_echantillon))
echantillon.createOrReplaceTempView("echantillon")
val echantillon_msg = spark.sql("select concat( case when qualite=0 then 'negatif' else 'positif' end, ' ', commentaire) as msg from echantillon")
val echantillonRDD = echantillon_msg.rdd
val hashingTF = new HashingTF(20000)


commentaires_negatifs = [Id_comment: string, commentaire: string ... 1 more field]
commentaires_positifs = [Id_comment: string, commentaire: string ... 1 more field]
commentaires_positifs_nbre = 188765
commentaires_negatifs_nbre = 3979
taille_echantillon = 3979
echantillon = [Id_comment: string, commentaire: string ... 1 more field]
echantillon_msg = [msg: string]
echantillonRDD = MapPartitionsRDD[28] at rdd at <console>:55
hashingTF = org.apache.spark.mllib.feature.HashingTF@5899d9bb


org.apache.spark.mllib.feature.HashingTF@5899d9bb

In [3]:
println("Commentaires                  = "+commentaires.count())
println("   dont commentaires négatifs = "+commentaires_negatifs.count())
println("   dont commentaires positifs = "+commentaires_positifs.count())
println("Taille echantillon = "+echantillonRDD.count())


Commentaires                  = 192744
   dont commentaires négatifs = 3979
   dont commentaires positifs = 188765
Taille echantillon = 7958                                                       


In [4]:
val echantillon_reformat = echantillonRDD.map(
  row =>{
    Try{
      val msg = row.toString.toLowerCase()
      var isHappy:Int = 0
      if(msg.contains("positif ")){
        isHappy = 1
      }else if(msg.contains("negatif ")){
        isHappy = 0
      }
      var msgSanitized = msg.replaceAll("positif ", "")
      msgSanitized = msgSanitized.replaceAll("negatif ","")
      //Return a tuple
      (isHappy, msgSanitized.split(" ").toSeq)
    }
  }
)

var echantillon_ok = echantillon_reformat.filter((_.isSuccess)).map(_.get)
val echantillon_transfo = echantillon_ok.map(
  t => (t._1, hashingTF.transform(t._2)))
  .map(x => new LabeledPoint((x._1).toDouble, x._2))

var sample = echantillon_ok.take(1000).map(
  t => (t._1, hashingTF.transform(t._2), t._2))
  .map(x => (new LabeledPoint((x._1).toDouble, x._2), x._3))

val splits = echantillon_transfo.randomSplit(Array(0.7, 0.3))
val (trainingData, validationData) = (splits(0), splits(1))


echantillon_reformat = MapPartitionsRDD[47] at map at <console>:47
echantillon_ok = MapPartitionsRDD[49] at map at <console>:65
echantillon_transfo = MapPartitionsRDD[51] at map at <console>:68
sample = 


Array(((1.0,(20000,[354,637,1113,1420,1566,2703,3680,4852,6061,6629,7604,8431,8809,9927,11259,11457,12061,13314,14489,14592,16025,18274,19003],[2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])),WrappedArray([hôte, extrêmement, attentif, et, très, attaché, à, remettre, en, ordre, les, petits, souc...


[((1.0,(20000,[354,637,1113,1420,1566,2703,3680,4852,6061,6629,7604,8431,8809,9927,11259,11457,12061,13314,14489,14592,16025,18274,19003],[2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])),WrappedArray([hôte, extrêmement, attentif, et, très, attaché, à, remettre, en, ordre, les, petits, soucis, (mineurs), qui, ont, pu, apparaître.
studio, sympa, et, très, bien, placé

a, recommander, !])), ((1.0,(20000,[354,1157,1566,2718,3982,5501,5787,5913,6129,6772,6948,7678,8749,8809,9232,9950,10824,11259,11735,12585,12597,12616,12627,13858,13896,14390,14489,16046,16324,17213,17834,18535,18700,18747,19637],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])),WrappedArray([aucun, problème, durant, le, séjour., 
vincent, fût, toujours, à, l'heure, ainsi, que, disponible, pour, répondre, à, mes, questions.
le, logement, était, parfaitement, fonctionnel, et, très, cozy

In [5]:
//CONSTRUCTION DU MODELE

val boostingStrategy = BoostingStrategy.defaultParams("Classification")
boostingStrategy.setNumIterations(20) //number of passes over our training data
boostingStrategy.treeStrategy.setNumClasses(2) //We have two output classes: happy and sad
boostingStrategy.treeStrategy.setMaxDepth(5)

val model = GradientBoostedTrees.train(trainingData, boostingStrategy)

//EVALUATION DU MODELE
var labelAndPredsTrain = trainingData.map { point =>
  val prediction = model.predict(point.features)
  Tuple2(point.label, prediction)
}

var labelAndPredsValid = validationData.map { point =>
  val prediction = model.predict(point.features)
  Tuple2(point.label, prediction)
}


[Stage 615:>                                                        (0 + 2) / 2]

boostingStrategy = BoostingStrategy(org.apache.spark.mllib.tree.configuration.Strategy@1b221496,org.apache.spark.mllib.tree.loss.LogLoss$@49634ca5,20,0.1,0.001)
model = 
labelAndPredsTrain = MapPartitionsRDD[594] at map at <console>:54
labelAndPredsValid = MapPartitionsRDD[595] at map at <console>:59


MapPartitionsRDD[595] at map at <console>:59

In [6]:
// Save and load model
model.save(sc, "modele/myGradientBoostingClassificationModel")


In [7]:
//APPLICATION DU MODELE SUR LE JEU D'APPRENTISSAGE
val results = labelAndPredsTrain.collect()


var Total_Positifs = 0
var Total_Negatifs = 0
var Correct_Positifs = 0
var Correct_Negatifs = 0
results.foreach(
  r => {
    if (r._1 == 1) {
      Total_Positifs += 1
    } else if (r._1 == 0) {
      Total_Negatifs += 1
    }
    if (r._1 == 1 && r._2 ==1) {
      Correct_Positifs += 1
    } else if (r._1 == 0 && r._2 == 0) {
      Correct_Negatifs += 1
    }
  }
)
println("messages négatifs dans le jeu de test : " + Total_Negatifs + " messages positifs : " + Total_Positifs)
println("% correct positifs: " + Correct_Positifs.toDouble/Total_Positifs)
println("% correct négatifs: " + Correct_Negatifs.toDouble/Total_Negatifs)

val testErr = labelAndPredsTrain.filter(r => r._1 != r._2).count.toDouble / trainingData.count()
println("Erreur commise sur le jeu d'apprentissage : " + testErr)



messages négatifs dans le jeu de test : 2714 messages positifs : 2815           
% correct positifs: 0.8895204262877442
% correct négatifs: 0.8327192336035372
Erreur commise sur le jeu d'apprentissage : 0.13836136733586543


results = Array((1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,0.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,0.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,0.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,0.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,0.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0...


[(1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,0.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,0.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,0.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,0.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,0.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,0.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,0.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0)

In [8]:
//APPLICATION DU MODELE SUR LE JEU DE VALIDATION
val results = labelAndPredsValid.collect()

var Total_Positifs = 0
var Total_Negatifs = 0
var Correct_Positifs = 0
var Correct_Negatifs = 0
results.foreach(
  r => {
    if (r._1 == 1) {
      Total_Positifs += 1
    } else if (r._1 == 0) {
      Total_Negatifs += 1
    }
    if (r._1 == 1 && r._2 ==1) {
      Correct_Positifs += 1
    } else if (r._1 == 0 && r._2 == 0) {
      Correct_Negatifs += 1
    }
  }
)
println("messages négatifs dans le jeu de validation : " + Total_Negatifs + " messages positifs : " + Total_Positifs)
println("% correct positifs : " + Correct_Positifs.toDouble/Total_Positifs)
println("% correct négatifs : " + Correct_Negatifs.toDouble/Total_Negatifs)

val testErr = labelAndPredsValid.filter(r => r._1 != r._2).count.toDouble / validationData.count()
println("Erreur commise sur le jeu de validation : " + testErr)


messages négatifs dans le jeu de validation : 1224 messages positifs : 1205
% correct positifs : 0.8224066390041493
% correct négatifs : 0.7802287581699346
Erreur commise sur le jeu de validation : 0.1988472622478386                    


results = Array((1.0,1.0), (1.0,0.0), (1.0,1.0), (1.0,0.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,0.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,0.0), (1.0,0.0), (1.0,1.0), (1.0,1.0), (1.0,0.0), (1.0,0.0), (1.0,0.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,0.0), (1.0,0.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0...


[(1.0,1.0), (1.0,0.0), (1.0,1.0), (1.0,0.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,0.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,0.0), (1.0,0.0), (1.0,1.0), (1.0,1.0), (1.0,0.0), (1.0,0.0), (1.0,0.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,0.0), (1.0,0.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,0.0), (1.0,1.0), (1.0,1.0), (1.0,0.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,0.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0), (1.0,1.0)

In [19]:
//COMPARAISON ENTRE VALEURS PREDITE ET REELLE SUR UN ECHANTILLON
val predictions = sample.map { point =>
  val prediction = model.predict(point._1.features)
  (point._1.label, prediction, point._2)
}

predictions.take(1000).foreach(x => if (x._1!=x._2) {println("label: " + x._1 + " prediction: " + x._2 + " text: " + x._3.mkString(" "))})


label: 1.0 prediction: 0.0 text: [l'appartement est idéalement situé. il n'y a pas de bruit, ce qui est appréciable à paris. il y a beaucoup de poussière dans l'appartement. désagréable, surtout pour quelqu'un qui y est très sensible. tout le centre de paris peut se faire à pied à partir de l'appartement, formidable. ]
label: 1.0 prediction: 0.0 text: [j'ai choisi cet apprt car j'ai participé au airbnb open à la villette - 15 min à pied le long du canal. rencontre extraordinaire, spontanée et magique entre 5000 personnes de 110 nationalités. un nouveau monde était en train de naître ... celui du partage d'idées, d'émotions, de visions - un nouveau paradigme planétaire dans le quel tout le monde pouvait être impliqué - quelle belle énéergie entre nous tous. merci airbnb pour ces deux premiers jours... puis le vendredi 13 nov s'est imposé et nous a tous secoué. 

sébastien est venu me chercher devant la porte de l'immeuble selon notre entente. 
la cour de l'immeuble est grande et tranqui

predictions = 


Array((1.0,1.0,WrappedArray([hôte, extrêmement, attentif, et, très, attaché, à, remettre, en, ordre, les, petits, soucis, (mineurs), qui, ont, pu, apparaître.
studio, sympa, et, très, bien, placé
a, recommander, !])), (1.0,1.0,WrappedArray([aucun, problème, durant, le, séjour., "
vincent", fût, toujours, à, l'heure, ainsi, que, disponible, pour, répondre, à, mes, questions.
le, logement, était, parfaitement, fonctionnel, et, très, cozy.
situé, dans, le, 6e,, ce, qui, ajoute, bien, sûr, un, cadre, agréable.
a, recommander.])), (1.0,1.0,WrappedArray([un, accueil, chaleureux, de, kathy, dans, cet, appartement, agréable, et, très, bien, placé, dans, paris, et, une, organisation, très, professionnelle, et, attentive, de, la, part, de, cobb...


[(1.0,1.0,WrappedArray([hôte, extrêmement, attentif, et, très, attaché, à, remettre, en, ordre, les, petits, soucis, (mineurs), qui, ont, pu, apparaître.
studio, sympa, et, très, bien, placé

a, recommander, !])), (1.0,1.0,WrappedArray([aucun, problème, durant, le, séjour., 
vincent, fût, toujours, à, l'heure, ainsi, que, disponible, pour, répondre, à, mes, questions.
le, logement, était, parfaitement, fonctionnel, et, très, cozy.
situé, dans, le, 6e,, ce, qui, ajoute, bien, sûr, un, cadre, agréable.
a, recommander.])), (1.0,1.0,WrappedArray([un, accueil, chaleureux, de, kathy, dans, cet, appartement, agréable, et, très, bien, placé, dans, paris, et, une, organisation, très, professionnelle, et, attentive, de, la, part, de, cobblestone., une, adresse, à, recommander.])), (1.0,1.0,WrappedArray([j'ai, été, accueilli, en, debut, d'après, midi, au, studio, par, la, sympathique, katia, qui, m'a, fourni, tous, les, renseignements, nécessaires, à, mon, court, sejour., je, l'ai, par, ailleurs,

In [20]:
predictions.foreach(x => if (x._1!=x._2) {println(x._1 + "#" + x._2 + "#" + x._3.mkString(" "))})


1.0#0.0#[l'appartement est idéalement situé. il n'y a pas de bruit, ce qui est appréciable à paris. il y a beaucoup de poussière dans l'appartement. désagréable, surtout pour quelqu'un qui y est très sensible. tout le centre de paris peut se faire à pied à partir de l'appartement, formidable. ]
1.0#0.0#[j'ai choisi cet apprt car j'ai participé au airbnb open à la villette - 15 min à pied le long du canal. rencontre extraordinaire, spontanée et magique entre 5000 personnes de 110 nationalités. un nouveau monde était en train de naître ... celui du partage d'idées, d'émotions, de visions - un nouveau paradigme planétaire dans le quel tout le monde pouvait être impliqué - quelle belle énéergie entre nous tous. merci airbnb pour ces deux premiers jours... puis le vendredi 13 nov s'est imposé et nous a tous secoué. 

sébastien est venu me chercher devant la porte de l'immeuble selon notre entente. 
la cour de l'immeuble est grande et tranquille mais pas allumée la nuit. 

le petit studio es